# 02.04 - Buenas Prácticas en Python

**Autor:** Miguel Angel Vazquez Varela  
**Nivel:** Fundamentos  
**Tiempo estimado:** 30 min

---

## ¿Qué aprenderás?

- Escribir código limpio siguiendo PEP 8
- Añadir type hints para mayor claridad
- Principios DRY y responsabilidad única
- Testear funciones con `assert` y `pytest`
- Organizar un proyecto Python correctamente

---

## 1. PEP 8: El estilo que todos entienden

PEP 8 es la guía de estilo oficial de Python. Seguirla hace que tu código sea legible por cualquier desarrollador.

In [1]:
# MAL: naming inconsistente, sin espacios, lineas largas
def calcSpeed(d,t,unit='kmh'):
    if unit=='kmh': return (d/(t/60))
    elif unit=='ms': return (d/(t/60))/3.6

# BIEN: nombres descriptivos, espacios, legible
def calculate_speed(distance_km: float, duration_min: float, unit: str = 'kmh') -> float:
    speed_kmh = distance_km / (duration_min / 60)

    if unit == 'kmh':
        return speed_kmh
    elif unit == 'ms':
        return speed_kmh / 3.6

### Reglas clave de PEP 8

| Elemento | Convención | Ejemplo |
|---|---|---|
| Variables y funciones | `snake_case` | `trip_duration` |
| Clases | `PascalCase` | `BikeStation` |
| Constantes | `UPPER_CASE` | `MAX_SPEED_KMH` |
| Privados | `_prefijo` | `_internal_counter` |
| Longitud de línea | máx. 79 chars | — |
| Espacios en blanco | 2 líneas entre funciones | — |

In [2]:
# Constantes al inicio del archivo o módulo
MAX_TRIP_DURATION_MIN = 120
MIN_TRIP_DISTANCE_KM = 0.1
VALID_BIKE_TYPES = ('electric', 'classic')


def is_valid_trip(distance_km: float, duration_min: float) -> bool:
    """Valida si un viaje cumple los criterios mínimos."""
    return (
        distance_km >= MIN_TRIP_DISTANCE_KM
        and duration_min <= MAX_TRIP_DURATION_MIN
    )


print(is_valid_trip(2.5, 30))   # True
print(is_valid_trip(0.05, 30))  # False (distancia mínima no alcanzada)

True
False


---

## 2. Type Hints: código que se documenta solo

Los type hints no son obligatorios en Python, pero aclaran qué espera y retorna cada función.

In [3]:
from typing import Optional

# Sin type hints: ¿qué tipo espera 'trips'? ¿qué retorna?
def get_average(trips):
    return sum(trips) / len(trips)


# Con type hints: todo queda claro
def get_average_duration(trips: list[float]) -> float:
    """Calcula la duración media de una lista de viajes."""
    return sum(trips) / len(trips)


# Optional indica que el valor puede ser None
def find_station(station_id: int, stations: list[dict]) -> Optional[dict]:
    """Busca una estación por ID. Retorna None si no existe."""
    for station in stations:
        if station['id'] == station_id:
            return station
    return None


stations = [
    {'id': 1, 'name': 'Sol', 'bikes': 12},
    {'id': 2, 'name': 'Atocha', 'bikes': 5},
]

result = find_station(1, stations)
print(result)

result = find_station(99, stations)
print(result)  # None

{'id': 1, 'name': 'Sol', 'bikes': 12}
None


In [4]:
# Type hints mas complejos
def classify_trips(durations: list[float]) -> dict[str, list[float]]:
    """
    Clasifica viajes en grupos por duración.

    Parameters
    ----------
    durations : list[float]
        Lista de duraciones en minutos

    Returns
    -------
    dict[str, list[float]]
        Diccionario con claves 'corto', 'medio', 'largo'
    """
    result: dict[str, list[float]] = {'corto': [], 'medio': [], 'largo': []}

    for d in durations:
        if d < 10:
            result['corto'].append(d)
        elif d <= 30:
            result['medio'].append(d)
        else:
            result['largo'].append(d)

    return result


trips = [5, 12, 35, 8, 25, 60, 15]
classified = classify_trips(trips)
for category, values in classified.items():
    print(f"{category}: {values}")

corto: [5, 8]
medio: [12, 25, 15]
largo: [35, 60]


---

## 3. DRY: Don't Repeat Yourself

Si copias y pegas código, algo está mal. Extrae la lógica repetida a una función.

In [5]:
# MAL: lógica duplicada
def report_madrid(trips: list[float]) -> None:
    total = sum(trips)
    avg = total / len(trips)
    print(f"Madrid - Total: {total:.1f} min | Media: {avg:.1f} min")

def report_barcelona(trips: list[float]) -> None:
    total = sum(trips)
    avg = total / len(trips)
    print(f"Barcelona - Total: {total:.1f} min | Media: {avg:.1f} min")


# BIEN: una sola función parametrizada
def report_city(city: str, trips: list[float]) -> None:
    """Imprime resumen de viajes para una ciudad."""
    total = sum(trips)
    avg = total / len(trips)
    print(f"{city} - Total: {total:.1f} min | Media: {avg:.1f} min")


report_city('Madrid', [12, 25, 8, 45])
report_city('Barcelona', [18, 30, 10, 22])

Madrid - Total: 90.0 min | Media: 22.5 min
Barcelona - Total: 80.0 min | Media: 20.0 min


### Responsabilidad única

Cada función debe hacer **una sola cosa**. Si describes una función con "y...", probablemente hace demasiado.

In [6]:
# MAL: esta función calcula Y filtra Y muestra
def process_trips_bad(trips: list[dict]) -> None:
    valid = [t for t in trips if t['duration'] > 0 and t['distance'] > 0]
    speeds = [t['distance'] / (t['duration'] / 60) for t in valid]
    avg = sum(speeds) / len(speeds)
    print(f"Velocidad media: {avg:.2f} km/h")


# BIEN: separado en responsabilidades claras
def filter_valid_trips(trips: list[dict]) -> list[dict]:
    """Elimina viajes con datos inválidos."""
    return [t for t in trips if t['duration'] > 0 and t['distance'] > 0]


def calculate_speeds(trips: list[dict]) -> list[float]:
    """Calcula la velocidad en km/h de cada viaje."""
    return [t['distance'] / (t['duration'] / 60) for t in trips]


def average(values: list[float]) -> float:
    """Calcula la media de una lista de valores."""
    return sum(values) / len(values)


# Composición limpia
raw_trips = [
    {'distance': 3.5, 'duration': 20},
    {'distance': 0, 'duration': 5},   # inválido
    {'distance': 5.0, 'duration': 30},
]

valid_trips = filter_valid_trips(raw_trips)
speeds = calculate_speeds(valid_trips)
avg_speed = average(speeds)

print(f"Velocidad media: {avg_speed:.2f} km/h")

Velocidad media: 10.25 km/h


---

## 4. Testing: verifica que tu código funciona

Probar el código evita que cambios futuros rompan funcionalidades ya implementadas.

In [7]:
# La forma más simple: assert
def calculate_speed(distance_km: float, duration_min: float) -> float:
    """Calcula velocidad en km/h."""
    if duration_min <= 0:
        raise ValueError("La duración debe ser mayor que 0")
    return distance_km / (duration_min / 60)


# Tests con assert
assert calculate_speed(10, 60) == 10.0,  "10 km en 60 min = 10 km/h"
assert calculate_speed(5, 30) == 10.0,   "5 km en 30 min = 10 km/h"
assert calculate_speed(1, 6) == 10.0,    "1 km en 6 min = 10 km/h"

print("Todos los tests pasaron")

Todos los tests pasaron


In [8]:
# Testear que los errores se lanzan correctamente
import traceback

try:
    calculate_speed(5, 0)
    print("ERROR: debería haber lanzado ValueError")
except ValueError as e:
    print(f"Correcto, ValueError capturado: {e}")

Correcto, ValueError capturado: La duración debe ser mayor que 0


### Estructura de un test con pytest

En proyectos reales se usa `pytest`. La convención es crear archivos `test_*.py`:

```python
# test_trips.py
import pytest
from trips import calculate_speed, filter_valid_trips


def test_calculate_speed_basic():
    assert calculate_speed(10, 60) == 10.0


def test_calculate_speed_zero_duration():
    with pytest.raises(ValueError):
        calculate_speed(5, 0)


def test_filter_removes_invalid():
    trips = [
        {'distance': 3.5, 'duration': 20},
        {'distance': 0,   'duration': 5},
    ]
    result = filter_valid_trips(trips)
    assert len(result) == 1
    assert result[0]['distance'] == 3.5
```

Para ejecutar: `pytest test_trips.py -v`

---

## 5. Estructura de un proyecto Python

Organizar bien los archivos es tan importante como el código en sí.

```
mi_proyecto/
├── data/                  # Datos crudos (no se tocan)
│   ├── raw/
│   └── processed/
├── notebooks/             # Exploración y análisis
│   └── 01_exploracion.ipynb
├── src/                   # Código reutilizable
│   ├── __init__.py
│   ├── cleaning.py
│   └── analysis.py
├── tests/                 # Tests automatizados
│   └── test_cleaning.py
├── requirements.txt       # Dependencias
└── README.md              # Descripción del proyecto
```

**Regla de oro:** los notebooks son para explorar, el código reutilizable va en `src/`.

In [9]:
# requirements.txt: fija las versiones para reproducibilidad
# pandas==2.2.0
# numpy==1.26.4
# matplotlib==3.8.3

# Para generar el tuyo:
# pip freeze > requirements.txt

# Para instalar en otro entorno:
# pip install -r requirements.txt

print("Ver requirements.txt en la raíz del proyecto")

Ver requirements.txt en la raíz del proyecto


---

## Resumen

| Práctica | Qué evita | Cómo aplicarla |
|---|---|---|
| PEP 8 | Código ilegible | `snake_case`, espacios, 79 chars |
| Type hints | Ambigüedad de tipos | `def f(x: int) -> str:` |
| DRY | Duplicación de lógica | Extraer a funciones parametrizadas |
| Responsabilidad única | Funciones que hacen demasiado | Una función = una tarea |
| Testing | Regresiones al cambiar código | `assert` o `pytest` |
| Estructura de proyecto | Caos de archivos | Separar `data/`, `src/`, `tests/` |

---

## Ejercicio

Refactoriza el siguiente código aplicando todo lo aprendido: añade type hints, divide en funciones con responsabilidad única y escribe al menos 2 asserts.

In [10]:
# Código a refactorizar
def process(data):
    out = []
    for x in data:
        if x['dur'] > 0 and x['dist'] > 0:
            spd = x['dist'] / (x['dur'] / 60)
            out.append({'dist': x['dist'], 'dur': x['dur'], 'spd': round(spd, 2)})
    total = sum(i['spd'] for i in out)
    print('media:', total / len(out))
    return out

# Tu solución aquí

---

**Anterior:** [02.03 - Módulos y Paquetes](./02_03_modules_and_packages.ipynb)  
**Siguiente:** [03.01 - Arrays con NumPy](../03_numpy_essentials/03_01_arrays_basics.ipynb)